In [ ]:
!pip install lazypredict
!pip install tpot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.1/297.1 MB 4.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.4/87.4 kB 17.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 17.6 MB/s eta 0:00:00
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11954 sha256=136551dad60f9daff92b021c0f06c63c2f3047a8f591c7f114228c2a0f5cc7e6
  Stored in directory: /root/.cache/pip/wheels/48/8c/93/3afb1916772591fe6bcc25cdf8b1c5bdc362f0ec8e2f0fd413
Successfully built stopit

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from lazypredict.Supervised import LazyRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, RepeatedKFold
from sklearn.metrics import make_scorer, mean_squared_error
from tpot import TPOTRegressor
from sklearn.linear_model import RidgeCV
from sklearn.pipeline import make_pipeline, make_union
from tpot.builtins import StackingEstimator
from xgboost import XGBRegressor
from tpot.export_utils import set_param_recursive
from sklearn.linear_model import SGDRegressor

In [ ]:
test_df = pd.read_csv("/work/test_preprocessed.csv")
train_df = pd.read_csv("/work/train_preprocessed.csv")

In [ ]:
test_df.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,BsmtQual,BsmtFinSF1,...,MiscVal,MoSold,Neighborhood_0,Neighborhood_1,Neighborhood_2,Neighborhood_3,Neighborhood_4,MasVnrType_0,MasVnrType_1,MasVnrType_2
0,-0.87,0.46,-13.16,-0.80,-0.34,-1.16,-0.57,-0.69,-0.83,0.05,...,-0.19,-0.12,-0.55,1.06,1.11,-0.79,-0.86,-0.10,0.68,-0.80
1,-0.87,0.51,-13.13,-0.07,-0.44,-1.30,0.03,-0.69,-0.83,1.05,...,-0.19,-0.12,-0.55,1.06,1.11,-0.79,-0.86,-0.10,-1.47,1.24
2,0.07,0.19,-13.10,-0.80,0.85,0.64,-0.57,-0.69,0.64,0.76,...,-0.19,-1.23,1.82,-0.94,-0.90,1.26,-0.86,-0.10,0.68,-0.80
3,0.07,0.37,-13.13,-0.07,0.89,0.64,-0.46,-0.69,-0.83,0.35,...,-0.19,-0.12,1.82,-0.94,-0.90,1.26,-0.86,-0.10,-1.47,1.24
4,1.49,-1.22,-13.06,1.37,0.69,0.35,-0.57,1.05,0.64,-0.40,...,-0.19,-1.97,1.82,-0.94,-0.90,1.26,1.16,-0.10,0.68,-0.80


In [ ]:
train_df.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,BsmtQual,BsmtFinSF1,...,MoSold,Neighborhood_0,Neighborhood_1,Neighborhood_2,Neighborhood_3,Neighborhood_4,MasVnrType_0,MasVnrType_1,MasVnrType_2,SalePrice
0,0.07,-0.22,-0.13,0.65,1.05,0.88,0.51,1.05,0.64,0.58,...,-1.60,-0.55,-0.94,-0.90,-0.79,1.16,-0.10,-1.47,1.24,12.25
1,-0.87,0.46,0.11,-0.07,0.16,-0.43,-0.57,-0.69,0.64,1.17,...,-0.49,-0.55,-0.94,-0.90,1.26,-0.86,-0.10,0.68,-0.80,12.11
2,0.07,-0.08,0.42,0.65,0.98,0.83,0.33,1.05,0.64,0.09,...,0.99,-0.55,-0.94,-0.90,-0.79,1.16,-0.10,-1.47,1.24,12.32
3,0.31,-0.45,0.10,0.65,-1.86,-0.72,-0.57,-0.69,-0.83,-0.50,...,-1.60,-0.55,-0.94,-0.90,1.26,1.16,-0.10,0.68,-0.80,11.85
4,0.07,0.64,0.88,1.37,0.95,0.73,1.37,1.05,0.64,0.46,...,2.10,-0.55,-0.94,1.11,-0.79,-0.86,-0.10,-1.47,1.24,12.43


In [ ]:
X_train = train_df.drop(["SalePrice"], axis=1)
y_train = train_df["SalePrice"]

In [ ]:
X_test = test_df

In [ ]:
X_train_tuning, X_test_tuning,y_train_tuning, y_test_tuning =  train_test_split(X_train, y_train, random_state=42, test_size=20)

In [ ]:
reg = LazyRegressor(verbose=0,ignore_warnings=True, custom_metric=None)
models = reg.fit(X_train_tuning, X_test_tuning,y_train_tuning, y_test_tuning)


100%|██████████| 42/42 [02:13<00:00,  3.18s/it][LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000681 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3021
[LightGBM] [Info] Number of data points in the train set: 1440, number of used features: 41
[LightGBM] [Info] Start training from score 12.025114



In [ ]:
models

(                               Adjusted R-Squared  R-Squared  RMSE  Time Taken
 Model                                                                         
 KernelRidge                                677.30    -853.27 12.06        0.49
 GaussianProcessRegressor                   488.66    -614.99 10.24        0.60
 Lars                                         4.65      -3.61  0.89        0.20
 MLPRegressor                                 3.29      -1.89  0.70        7.00
 Lasso                                        1.82      -0.03  0.42        0.10
 LassoLars                                    1.82      -0.03  0.42        0.13
 ElasticNet                                   1.82      -0.03  0.42        0.01
 DummyRegressor                               1.82      -0.03  0.42        0.02
 QuantileRegressor                            1.81      -0.02  0.42      114.40
 DecisionTreeRegressor                        1.20       0.75  0.21        0.18
 AdaBoostRegressor                      

In [ ]:
# Average CV score on the training set was: -0.13835166977927088
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=RidgeCV()),
    XGBRegressor(learning_rate=0.1, max_depth=5, min_child_weight=16, n_estimators=100, n_jobs=1, objective="reg:squarederror", subsample=0.4, verbosity=0)
)
# Fix random state for all the steps in exported pipeline
set_param_recursive(exported_pipeline.steps, 'random_state', 1)

exported_pipeline.fit(X_train, y_train)
results = exported_pipeline.predict(X_test)

In [ ]:
# Define the hyperparameters grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.1, 0.01, 0.001],
}
# Define RMSE as a scorer
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

rmse_scorer = make_scorer(rmse, greater_is_better=False)
# Instantiate XGBRegressor
# Average CV score on the training set was: -0.13835166977927088
xgb = XGBRegressor(min_child_weight=16, n_jobs=1, objective="reg:squarederror", subsample=0.4, verbosity=0)
# Instantiate GridSearchCV with RMSE as the scoring function
grid_search = GridSearchCV(xgb, param_grid, cv=5, scoring=rmse_scorer, n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(X_train_tuning, y_train_tuning)

# Print the best hyperparameters
print("Best hyperparameters:", grid_search.best_params_)

# Evaluate the best model on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_tuning)
test_rmse = rmse(y_test_tuning, y_pred)
print("Test set RMSE:", test_rmse)

Best hyperparameters: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}
Test set RMSE: 0.09312199273993274


In [ ]:
import pickle
# save the model to disk
filename = 'finalized_model.pkl'
pickle.dump(best_model, open(filename, 'wb'))

In [ ]:
# Average CV score on the training set was: -0.13835166977927088
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=RidgeCV()),
   best_model
)
# Fix random state for all the steps in exported pipeline
set_param_recursive(exported_pipeline.steps, 'random_state', 1)

exported_pipeline.fit(X_train_tuning, y_train_tuning)
y_pred = exported_pipeline.predict(X_test_tuning)
test_rmse = rmse(y_test_tuning, y_pred)
print("Test set RMSE:", test_rmse)

Test set RMSE: 0.11468274585162457


In [ ]:
# Define the hyperparameters grid
param_grid = {
    'alpha': [0.0001, 0.001, 0.01, 0.1, 1.0],  # Regularization parameter
    'max_iter': [1000, 2000, 3000],  # Maximum number of iterations
}

# Instantiate GridSearchCV
grid_search = GridSearchCV(SGDRegressor(), param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(X_train_tuning, y_train_tuning)

# Print the best hyperparameters
print("Best hyperparameters:", grid_search.best_params_)

# Evaluate the best model on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_tuning)
test_rmse = rmse(y_test_tuning, y_pred)
print("Test set RMSE:", test_rmse)

Best hyperparameters: {'alpha': 0.1, 'max_iter': 3000}
Test set RMSE: 0.1229791374427065


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=79960be5-3388-46e3-82a2-0b0df5ae3fd3' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>